In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from litQeval.eval_utils import *
import plotly.express as px
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [16]:
baseline = 'Robotic Arthroplasty'
predicted = """
"digital surgery" OR "surgical navigation" OR "haptic feedback" OR "robot-assisted surgery" OR "prosthetic devices" OR "computer-assisted surgery" OR "surgical workflow" OR (robotic arthroplasty) OR 
(("clinical trials" OR "rehabilitation" OR "recovery time" OR "3d imaging" OR "patient satisfaction" OR "minimally invasive surgery" OR "surgical safety" OR "advanced imaging techniques" OR "orthopedic surgery" OR "total hip arthroplasty" OR "total knee arthroplasty" OR "biomechanics" OR "surgeon training" OR "motion planning" OR "joint replacement" OR "surgical precision" OR "robotic surgery" OR "implant technology" OR "surgery simulation" OR "surgical robotics") AND ("Arthoplasty"))
"""

data = get_data(baseline, predicted)
core_pubs = data["core_pubs"]
core_mean_embedding = data["core_mean_embedding"]
baseline_pubs = data["baseline_pubs"]
predicted_pubs = data["predicted_pubs"]
baseline_vs = data["baseline_vs"]
predicted_vs = data["predicted_vs"]
core_vs = data["core_vs"]
predicted_embeddings = np.array([embedding for embedding in predicted_vs.get(include=["embeddings"])["embeddings"]])
baseline_embeddings = np.array([embedding for embedding in baseline_vs.get(include=["embeddings"])["embeddings"]])
core_embeddings = np.squeeze([core_vs.get(i,include=["embeddings"])["embeddings"] for i in core_pubs])
core_mean_embedding.reshape(1, -1).shape, predicted_embeddings.shape, baseline_embeddings.shape


((1, 1536), (22892, 1536), (2151, 1536))

In [17]:
# baseline
cosine_sim = cosine_similarity(core_mean_embedding, baseline_embeddings).flatten()
baseline_pubs["similarity"] = cosine_sim

core_pubs_in_baseline = baseline_pubs[baseline_pubs["id"].isin(core_pubs)]
threshold = core_pubs_in_baseline["similarity"].min()
relevent_baseline_pubs = baseline_pubs[baseline_pubs["similarity"] >= threshold].copy()
print(f"Number of core publications in the baseline: {core_pubs_in_baseline.shape[0]}")
print(f"Number of relevant publications in the baseline: {relevent_baseline_pubs.shape[0]}")

Number of core publications in the baseline: 45
Number of relevant publications in the baseline: 1279


In [18]:
# predicted
cosine_sim = cosine_similarity(core_mean_embedding, predicted_embeddings).flatten()
predicted_pubs["similarity"] = cosine_sim

core_pubs_in_predicted = predicted_pubs[predicted_pubs["id"].isin(core_pubs)]
threshold = core_pubs_in_predicted["similarity"].min()
relevant_predicted_pubs = predicted_pubs[predicted_pubs["similarity"] >= threshold].copy()
print(f"Number of core publications in the predicted: {core_pubs_in_predicted.shape[0]}")
print(f"Number of relevant publications in the predicted: {relevant_predicted_pubs.shape[0]}")

Number of core publications in the predicted: 45
Number of relevant publications in the predicted: 1403


### Cosine Similarity Measure

In [19]:
recall = evaluate_recall(core_pubs, baseline_pubs, predicted_pubs)
# semnatic precision: every element that is more similar than the least similar core publication is considered relevant
# relevant_predicted_pubs: publications that are more similar than the least similar core publication.
pred_precision = relevant_predicted_pubs.shape[0] / predicted_pubs.shape[0] # total number of found publications
baseline_precision = (relevent_baseline_pubs.shape[0] / baseline_pubs.shape[0]) if baseline_pubs.shape[0] > 0 else 0
pred_f2 = fscore(pred_precision, recall["predicted_recall"], 2)
baseline_f2 = fscore(baseline_precision, recall["baseline_recall"], 2)
df = pd.DataFrame({
    "Semantic Precision": [pred_precision, baseline_precision],
    "Recall": [recall["predicted_recall"], recall["baseline_recall"]],
    "Semantic F2": [pred_f2, baseline_f2]
}, index=["Predicted", "Baseline"])
df

,Semantic Precision,Recall,Semantic F2
Predicted,0.061288,0.957447,0.243971
Baseline,0.594607,0.957447,0.853306


### Minimum Volume Enclosing Ellipsoid MMVE

In [20]:
import numpy.linalg as la

def mvee(points, tol=0.0001):
    """
    Finds the ellipse equation in "center form"
    (x-c).T * A * (x-c) = 1
    """
    N, d = points.shape
    Q = np.column_stack((points, np.ones(N))).T
    err = tol+1.0
    u = np.ones(N)/N
    while err > tol:
        # assert u.sum() == 1 # invariant
        X = np.dot(np.dot(Q, np.diag(u)), Q.T)
        M = np.diag(np.dot(np.dot(Q.T, la.inv(X)), Q))
        jdx = np.argmax(M)
        step_size = (M[jdx]-d-1.0)/((d+1)*(M[jdx]-1.0))
        new_u = (1-step_size)*u
        new_u[jdx] += step_size
        err = la.norm(new_u-u)
        u = new_u
    c = np.dot(u, points)
    A = la.inv(np.dot(np.dot(points.T, np.diag(u)), points)
               - np.multiply.outer(c, c))/d
    return A, c

A, c = mvee(core_embeddings)

In [21]:
base_is_inside = is_inside_ellipse(A, c, baseline_embeddings)
predicted_is_inside = is_inside_ellipse(A, c, predicted_embeddings)

100%|██████████| 22892/22892 [00:46<00:00, 491.58it/s]


In [22]:
mvve_prec_baseline = base_is_inside.sum() / len(base_is_inside)
mvve_prec_predicted = predicted_is_inside.sum() / len(predicted_is_inside)

mvve_df = pd.DataFrame({
    "MVVE Precision": [mvve_prec_predicted, mvve_prec_baseline],
    "Recall": [recall["predicted_recall"], recall["baseline_recall"]],
    "MVVE F2": [fscore(mvve_prec_predicted, recall["predicted_recall"], 2), fscore(mvve_prec_baseline, recall["baseline_recall"], 2)]
}, index=["Predicted", "Baseline"])
print(f"Baseline - Inside: {base_is_inside.sum()}")
print(f"Predicted - Inside: {predicted_is_inside.sum()}")
mvve_df

Baseline - Inside: 862
Predicted - Inside: 7560


,MVVE Precision,Recall,MVVE F2
Predicted,0.330246,0.957447,0.693884
Baseline,0.400744,0.957447,0.749273


In [31]:
results = pd.DataFrame({
    "Query": [predicted] + [baseline],
    "Recall": [recall["predicted_recall"], recall["baseline_recall"]],
    "Semantic Precision": [pred_precision, baseline_precision],
    "Semantic F2": [pred_f2, baseline_f2],
    "MVVE Precision": [mvve_prec_predicted, mvve_prec_baseline],
    "MVVE F2": [fscore(mvve_prec_predicted, recall["predicted_recall"], 2), fscore(mvve_prec_baseline, recall["baseline_recall"], 2)]
}, index=["Predicted", "Baseline"])

try:
    old_results = pd.read_excel("results.xlsx", index_col=0)
    results = pd.concat([old_results, results]).drop_duplicates(subset=["Query"]).round(3)
    results.to_excel("results.xlsx")
except FileNotFoundError:
    results.to_excel("results.xlsx")

display(results.sort_index())

,Query,Recall,Semantic Precision,Semantic F2,MVVE Precision,MVVE F2
Baseline,"""Cervical Myelopathy""",0.277,0.523,0.305,0.489,0.303
Baseline,"""Drones in Agriculture""",0.000,0.000,0.000,0.592,0.000
Baseline,Robotic Arthroplasty,0.957,0.595,0.853,0.401,0.749
Baseline,Soft Robotics,0.556,0.452,0.531,0.492,0.542
Baseline,Crop Yield Prediction,0.543,0.370,0.497,0.508,0.536
Predicted,"\n""spinal stenosis"" OR ""spinal surgery risks"" ...",0.723,0.316,0.575,0.438,0.640
Predicted,"\n""geospatial data"" OR ""aerial photography"" OR...",0.120,0.036,0.082,0.518,0.142
Predicted,"\n""digital surgery"" OR ""surgical navigation"" O...",0.957,0.061,0.244,0.330,0.694
Predicted,"\n""compliant materials"" OR (soft robotics) OR ...",0.722,0.212,0.487,0.451,0.644
Predicted,"\n""crop yield estimation"" OR ""crop simulation ...",0.652,0.106,0.321,0.528,0.623
